<a href="https://colab.research.google.com/github/OJB-Quantum/Notebooks-for-Ideas/blob/main/Onris_Layoff_Simulator_and_Oversight_Indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authored by Onri Jay Benally (2025)

Open Access (CC-BY-4.0)

In [1]:
# @title Cell 1: Setup and Installations
# Install CuPy, which is NumPy for the GPU. We specify the CUDA version for Colab.
!pip install cupy-cuda12x ipywidgets matplotlib seaborn --quiet

# Enable ipywidgets for interactive sliders in Colab
from google.colab import output
output.enable_custom_widget_manager()

import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, HBox, HTML, Layout
import warnings

plt.rcParams['figure.dpi'] = 200

# Suppress routine warnings for a cleaner output
warnings.filterwarnings('ignore')

# Check for GPU and print confirmation
try:
    gpu_device = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_device['name'].decode('utf-8')
    print(f"GPU successfully detected: {gpu_name}")
except cp.cuda.runtime.CUDARuntimeError as e:
    print(f"GPU not found. Please ensure you've selected a GPU runtime. Error: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00
GPU successfully detected: Tesla T4


In [5]:
# @title Cell 2: Layoff Risk Simulator
def run_layoff_simulation(
    num_employees, layoff_threshold, w_performance, w_salary,
    w_redundancy, w_network, f_org_health, f_macro_shock
):
    """
    Calculates layoff risk scores for a synthetic workforce on the GPU.
    """
    # 1. Generate Synthetic Employee Data on the GPU
    # Performance: Lower is worse (1-4, with 1 being bottom quartile). We invert it for the risk score.
    performance = 4 - cp.random.randint(1, 5, num_employees)
    # Salary vs Peers: Normalized from -0.5 to 0.5. Higher salary = higher risk score.
    salary_vs_peers = cp.random.rand(num_employees) - 0.5
    # Skill Redundancy: 1 if redundant, 0 if unique.
    skill_redundancy = cp.random.randint(0, 2, num_employees)
    # Internal Network Strength: Lower is worse (0-1). We invert it for the risk score.
    network_strength = 1 - cp.random.rand(num_employees)

    # 2. Calculate Layoff Risk Score (Vectorized GPU Operation)
    # This is the core calculation. All operations happen in parallel on the GPU.
    risk_score = (
        (performance * w_performance) +
        (salary_vs_peers * w_salary) +
        (skill_redundancy * w_redundancy) +
        (network_strength * w_network) +
        (f_org_health * -1) + # Poor health increases risk
        (f_macro_shock * -1)  # A macro shock increases risk
    )

    # Normalize score to be roughly between 0 and 100 for easier interpretation
    risk_score_normalized = (risk_score - risk_score.min()) / (risk_score.max() - risk_score.min()) * 100
    risk_score_np = cp.asnumpy(risk_score_normalized) # Move data to CPU for plotting

    # 3. Analyze and Plot Results
    high_risk_count = int(cp.sum(risk_score_normalized > layoff_threshold))
    high_risk_percentage = (high_risk_count / num_employees) * 100

    # Plotting
    plt.style.use('seaborn-v0_8-darkgrid')
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.histplot(risk_score_np, bins=50, kde=True, ax=ax, color='blue', alpha=0.5)
    ax.axvline(layoff_threshold, color='#c44e52', linestyle='--', linewidth=2, label=f'High-Risk Threshold ({layoff_threshold})')
    ax.set_title(f'Distribution of Layoff Risk Scores ({num_employees:,} Employees)', fontsize=16, fontweight='bold')
    ax.set_xlabel('Calculated Layoff Risk Score', fontsize=12)
    ax.set_ylabel('Number of Employees', fontsize=12)
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Add annotation for the high-risk group
    ax.text(
        0.98, 0.95,
        f'High-Risk Employees: {high_risk_count:,}\n({high_risk_percentage:.2f}%)',
        transform=ax.transAxes,
        fontsize=12,
        verticalalignment='top',
        horizontalalignment='right',
        bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5)
    )
    plt.show()

# --- Create Interactive Sliders ---
style = {'description_width': 'initial'}
layout = Layout(width='80%')

# Define the sliders for the simulation
sim_interactive = interactive(
    run_layoff_simulation,
    num_employees=IntSlider(value=500000, min=10000, max=5000000, step=10000, description='Number of Employees:', style=style, layout=layout),
    layoff_threshold=IntSlider(value=75, min=0, max=100, step=1, description='High-Risk Threshold:', style=style, layout=layout),
    w_performance=FloatSlider(value=0.4, min=0, max=1, step=0.05, description='Weight: Performance:', style=style, layout=layout),
    w_salary=FloatSlider(value=0.2, min=0, max=1, step=0.05, description='Weight: Salary vs Peers:', style=style, layout=layout),
    w_redundancy=FloatSlider(value=0.8, min=0, max=1, step=0.05, description='Weight: Skill Redundancy:', style=style, layout=layout),
    w_network=FloatSlider(value=0.3, min=0, max=1, step=0.05, description='Weight: Weak Network:', style=style, layout=layout),
    f_org_health=FloatSlider(value=0.5, min=-1, max=1, step=0.1, description='Shock: Org Health (-1=Poor, 1=Great):', style=style, layout=layout),
    f_macro_shock=FloatSlider(value=0.8, min=-1, max=1, step=0.1, description='Shock: Macro Economy (-1=Recession, 1=Boom):', style=style, layout=layout)
)

# --- Display the UI ---
sim_title = HTML("<h2> Part 1: CUDA-Accelerated Layoff Risk Simulator</h2>")
sim_description = HTML("<p>This model simulates layoff risk across a workforce. Adjust the weights and shock factors to see how risk changes. The heavy computation is done on the GPU.</p>")
display(VBox([sim_title, sim_description, sim_interactive]))

In [4]:
# @title Cell 3: Governance Oversight Indicator
from google.colab import output
from ipywidgets import interactive, IntSlider, VBox, HBox, HTML, Layout
from IPython.display import display

def display_oversight_indicator(layoff_rounds, survivor_turnover, engagement_drop):
    """
    Analyzes layoff frequency and impact to flag potential governance oversight.
    """
    flags = []
    risk_level = "LOW"
    risk_color = "#2ca02c" # Green
    recommendation = "Current workforce actions appear to be within standard strategic adjustments. Monitor employee morale and retention as a best practice."

    # --- Logic based on user notes ---
    if layoff_rounds == 2:
        risk_level = "MEDIUM"
        risk_color = "#ff7f0e" # Orange
        flags.append("<b>Second-Round Early-Warning:</b> Two rounds of layoffs in a year necessitate an immediate review of forecasting assumptions by audit, HR, and strategy committees.")
        recommendation = "Convene committee review to scrutinize forecasting models. A pre-emptive analysis of redeployment and reskilling options is highly recommended before considering any further action."

    if layoff_rounds >= 3:
        risk_level = "CRITICAL"
        risk_color = "#d62728" # Red
        flags.append("<b>Strategic Mis-forecasting:</b> Three or more rounds signal that revenue and demand models are likely flawed or that strategic planning has failed.")
        flags.append("<b>Investor Confidence Risk:</b> Serial layoffs are often interpreted by the market as mismanagement, potentially leading to P/E compression and analyst downgrades.")
        flags.append("<b>Regulatory Aggregation Risk (WARN Act):</b> Staggered cuts within a 90-day window can be aggregated, exposing the firm to significant legal and financial penalties for failure to provide adequate notice.")
        recommendation = "<b>Board Intervention Required.</b> Mandate a 24-month workforce planning model, a full cost-benefit analysis comparing further cuts to redeployment, and a formal Investor Relations briefing on talent-brand impact before authorizing any additional reductions."

    if survivor_turnover >= 30:
        flags.append(f"<b>Compounded Survivor Turnover ({survivor_turnover}%):</b> The current attrition rate among remaining employees is wiping out a significant portion of the initial payroll savings and indicates severe morale issues.")
        if risk_level == "LOW":
            risk_level = "MEDIUM"
            risk_color = "#ff7f0e"

    if engagement_drop >= 15:
        flags.append(f"<b>Engagement & Brand Erosion ({engagement_drop}% drop):</b> A significant drop in engagement scores predicts future productivity loss and harms the employer brand, making it harder and more expensive to attract top talent.")
        if risk_level == "LOW":
            risk_level = "MEDIUM"
            risk_color = "#ff7f0e"

    # --- HTML Output Formatting ---
    flags_html = "".join([f"<li style='margin-bottom: 8px;'>{flag}</li>" for flag in flags]) if flags else "<li>No major flags detected.</li>"

    # --- MODIFICATION START: Added 'color: #333;' to the main div and specific darker colors to headers ---
    output_html = f"""
    <div style='border: 2px solid {risk_color}; padding: 15px; border-radius: 8px; font-family: sans-serif; background-color: #f9f9f9; color: #333333;'>
        <h3 style='margin-top: 0; color: {risk_color};'>Oversight Risk Level: {risk_level}</h3>
        <hr style='border-top: 1px solid {risk_color};'>
        <h4 style='color: #111111;'>Triggered Flags:</h4>
        <ul style='padding-left: 20px;'>
            {flags_html}
        </ul>
        <h4 style='color: #111111;'>Board Recommendation:</h4>
        <p style='margin-bottom: 0;'>{recommendation}</p>
    </div>
    """
    # --- MODIFICATION END ---
    display(HTML(output_html))


# --- Create Interactive Sliders ---
style = {'description_width': 'initial'}
layout = Layout(width='80%')

indicator_interactive = interactive(
    display_oversight_indicator,
    layoff_rounds=IntSlider(value=1, min=0, max=5, step=1, description='Layoff Rounds (last 12 mos):', style=style, layout=layout),
    survivor_turnover=IntSlider(value=10, min=0, max=50, step=1, description='Voluntary Survivor Turnover (%):', style=style, layout=layout),
    engagement_drop=IntSlider(value=5, min=0, max=50, step=1, description='Employee Engagement Drop (%):', style=style, layout=layout)
)


# --- Display the UI ---
indicator_title = HTML("<h2> Part 2: Board Oversight & Governance Indicator</h2>")
indicator_description = HTML("<p>This tool assesses governance risk based on layoff frequency and its after-effects. If a company needs a <b>third round</b> of cuts within a year, it typically signals a failure of strategy, not agility.</p>")
display(VBox([indicator_title, indicator_description, indicator_interactive]))